# Examen 1 : Regresión - Correción

André Esteban Vera

## Parte Teórica

**1.¿Qué es un pipeline (flujo de pasos)?**

Un *pipeline* es básicamente una serie o cadena de pasos que se ejecutan en orden para automatizar un proceso. 
Nos ayuda a conectar desde la limpieza y preparación de la información hasta 
la construcción y evaluación de un modelo, todo dentro de una misma estructura.

**2.¿Cuál es el propósito de realizar regresiones? Explica las ventajas y desventajas de los dos planteamientos vistos en clase.**

La regresión busca entender y predecir cómo se relacionan las variables independientes (X) con la 
dependiente (Y).  
- **Sin penalización:** sencillo de interpretar, porque cada coeficiente refleja directamente el impacto de la variable. 
La parte negativa es que puede sobreajustarse y ser sensible al ruido.  
- **Con penalización:** ayuda a estabilizar los coeficientes, controlar la multicolinealidad y evitar sobreajuste. 
Lo malo es que interpretar los coeficientes ya no es tan directo y, a veces, se pierde algo de precisión.

**3.¿En qué consiste el proceso de escalamiento de factores?**

Es poner todas las variables en la misma escala, de forma que ninguna domine al modelo solo por tener valores numéricos más grandes.

**4.Explica el propósito de penalizar factores en una regresión.**

Se hace para que el modelo sea menos complejo y evite memorizar demasiado los datos de entrenamiento. 
La penalización castiga los valores grandes de los coeficientes (ya sea con cuadrados o valores absolutos).

**5.¿Cuál es la relación entre escalamiento y penalización?**

Si no escalamos antes, las penalizaciones tratarían de forma injusta a las variables de mayor magnitud. 
Por eso el escalamiento es un paso previo funadamental y necesario.

**6.Explica el concepto de una prueba de hipótesis.**

Es un procedimiento para evaluar, con datos de una muestra, si una afirmación sobre la población tiene sentido o no. 
Se plantean dos hipótesis (nula y alternativa) y, según la evidencia, decidimos si rechazamos la nula.

**7.Explica la interpretación de un p-value de una prueba de hipótesis que compara contra una media µ.**

El *p-value* nos dice qué tan raro sería obtener los datos si en realidad la hipótesis nula fuera cierta.  
Si el valor es menor al nivel de significancia, se rechaza H0. Ejemplo: al comparar la media de una muestra contra µ=50, 
si el p-value=0.01, significa que habría solo un 1% de probabilidad de ver esos datos si la media real fuera 50.

**8.Describe el propósito de realizar cross-validation.**

Sirve para comprobar qué tan bien generaliza un modelo. Dividimos los datos en varios subconjuntos, 
entrenamos y probamos en diferentes combinaciones, lo que reduce el riesgo de sobreajuste.

**9.Describe los pasos que seguirías al hacer un análisis exploratorio de datos. Justifica cada paso.**

los pasos del EDA son:
1. Cargar los datos y limpiarlos (nulos, duplicados, inconsistencias).  
2. Revisar qué tipo de variables tenemos (categóricas, numéricas, fechas).  
3. Obtener estadísticas descriptivas y detectar posibles outliers.  
4. Transformar variables si hace falta (crear *dummies*, escalar, etc.).  

**10.¿Qué es el teorema del límite central?**

El TLC dice que, aunque la población tenga cualquier distribución, si tomamos muchas muestras grandes 
y calculamos sus medias, la distribución de esas medias tenderá a una normal conforme el tamaño de la 
muestra crezca.


En esta parte anterior la verdad no supe exactamente cuales fueron mis errores aparte de la 2 y 7 que fueron en las únicas donde se me hizo comentario así que decidí hacer todas casi desde cero.

## 1) Preparación de datos

- Quitamos duplicados y nulos para evitar sesgos y errores en métricas.
- Documentamos cada transformación para que sea reproducible.

In [39]:
# Carga de datos
import pandas as pd
datos = pd.read_csv('imdb_top1000_lae.csv')
datos.head(5)

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,IMDB_Rating,Meta_score,Director,Gross,Drama,...,Fantasy,Family,Thriller,Romance,Sci-Fi,War,Music,Musical,Sport,History
0,0,The Shawshank Redemption,1994,A,142 min,9.3,80.0,Frank Darabont,"28,341,469",1,...,0,0,0,0,0,0,0,0,0,0
1,1,The Godfather,1972,A,175 min,9.2,100.0,Francis Ford Coppola,"134,966,411",1,...,0,0,0,0,0,0,0,0,0,0
2,2,The Dark Knight,2008,UA,152 min,9.0,84.0,Christopher Nolan,"534,858,444",1,...,0,0,0,0,0,0,0,0,0,0
3,3,The Godfather: Part II,1974,A,202 min,9.0,90.0,Francis Ford Coppola,"57,300,000",1,...,0,0,0,0,0,0,0,0,0,0
4,4,12 Angry Men,1957,U,96 min,9.0,96.0,Sidney Lumet,"4,360,000",1,...,0,0,0,0,0,0,0,0,0,0


### Limpieza base
Quitamos duplicados y filas con valores faltantes para que las métricas no se distorsionen. Este paso no cambia la distribución de los datos "buenos", solo elimina ruido.

In [40]:
datos = datos.drop_duplicates()
datos = datos.dropna()
datos.describe()

,Unnamed: 0,IMDB_Rating,Meta_score,Drama,Crime,Action,Biography,Western,Comedy,Adventure,...,Fantasy,Family,Thriller,Romance,Sci-Fi,War,Music,Musical,Sport,History
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,...,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.00000,714.000000,714.000000,714.000000
mean,518.572829,7.937115,77.158263,0.700280,0.198880,0.196078,0.123249,0.022409,0.225490,0.228291,...,0.077031,0.060224,0.138655,0.123249,0.078431,0.040616,0.04902,0.015406,0.023810,0.053221
std,295.848106,0.293278,12.401144,0.458456,0.399437,0.397307,0.328954,0.148113,0.418198,0.420026,...,0.266827,0.238068,0.345829,0.328954,0.269038,0.197538,0.21606,0.123248,0.152562,0.224632
min,0.000000,7.600000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,262.250000,7.700000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,526.500000,7.900000,78.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,777.750000,8.100000,86.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
max,997.000000,9.300000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


### Selección de variables
Quitamos columnas que no aportan al modelado o que complican innecesariamente la estructura.

In [41]:
cols_a_remover = ['Unnamed: 0','Certificate','Series_Title','Director', 'Gross']
datos = datos.drop(columns=cols_a_remover, errors='ignore')
list(datos.columns)[:10]

['Released_Year',
 'Runtime',
 'IMDB_Rating',
 'Meta_score',
 'Drama',
 'Crime',
 'Action',
 'Biography',
 'Western',
 'Comedy']

## 2) Transformaciones y verificaciones
- Estandarizamos el tipo de dato en `Released_Year` (algunas entradas no son año).  
- Revisamos estadísticos básicos para detectar valores extremos antes de modelar.

In [42]:
# Filtrar entradas no numéricas en Released_Year (por ejemplo, 'PG') y convertir a entero
datos = datos[datos['Released_Year'] != 'PG']
datos['Released_Year'] = pd.to_numeric(datos['Released_Year'], errors='coerce')
datos = datos.dropna(subset=['Released_Year'])
datos['Released_Year'] = datos['Released_Year'].astype(int)

# Runtime : converimos a int 
datos['Runtime'] = datos['Runtime'].astype(str).str.replace(' min', '', regex=False)
datos['Runtime'] = datos['Runtime'].astype(int)

genre_cols = [
    'Drama','Crime','Action','Biography','Western','Comedy','Adventure','Animation',
    'Horror','Mystery','Film-Noir','Fantasy','Family','Thriller','Romance','Sci-Fi',
    'War','Music','Musical','Sport','History'
]
genre_cols = [c for c in genre_cols if c in datos.columns]

for c in genre_cols:
    # Cualquier valor no nulo y distinto de 0 lo mapeamos a 1, lo demás a 0
    datos[c] = (datos[c].astype(str).str.strip().str.lower().isin(['1','true','yes','y','t']) | (pd.to_numeric(datos[c], errors='coerce') > 0)).astype(int)
datos.describe()

,Released_Year,Runtime,IMDB_Rating,Meta_score,Drama,Crime,Action,Biography,Western,Comedy,...,Fantasy,Family,Thriller,Romance,Sci-Fi,War,Music,Musical,Sport,History
count,713.000000,713.000000,713.000000,713.000000,713.00000,713.000000,713.000000,713.000000,713.000000,713.000000,...,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000,713.000000
mean,1995.736325,123.692847,7.937588,77.158485,0.69986,0.199158,0.196353,0.123422,0.022440,0.225806,...,0.077139,0.060309,0.138850,0.123422,0.078541,0.040673,0.049088,0.015428,0.023843,0.051893
std,18.598222,25.898509,0.293211,12.409849,0.45864,0.399648,0.397518,0.329152,0.148215,0.418406,...,0.266999,0.238225,0.346033,0.329152,0.269210,0.197671,0.216204,0.123333,0.152667,0.221968
min,1930.000000,72.000000,7.600000,28.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1987.000000,104.000000,7.700000,70.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2001.000000,120.000000,7.900000,78.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2010.000000,136.000000,8.100000,86.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2019.000000,238.000000,9.300000,100.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
# Librerías
import numpy as np
from sklearn.metrics import r2_score
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler

In [44]:
# Asignar variables predictoras y objetivo
target_col = 'IMDB_Rating'
num_cols = [c for c in ['Released_Year','Runtime','Meta_score'] if c in datos.columns]
X_cols = num_cols + genre_cols
X = datos[X_cols].copy()
y = datos[target_col].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.4, random_state=137)



In [45]:
# Escalamiento SOLO a numéricas; los géneros ya son 0/1
num_idx = [i for i, c in enumerate(X.columns) if c in num_cols]
scaler = StandardScaler()

# Copias para escalamiento
X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()

# Ajuste/transform a numéricas
X_train_scaled.iloc[:, num_idx] = scaler.fit_transform(X_train.iloc[:, num_idx])
X_test_scaled.iloc[:, num_idx] = scaler.transform(X_test.iloc[:, num_idx])

C:\Users\chino\AppData\Local\Temp\ipykernel_10868\2702799590.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.48314629 -0.06392043  0.70197297 -0.1186271   0.10019959 -3.23690736
  1.08491967  0.81138631 -0.28274711  0.70197297  0.97550633  0.81138631
  0.59255963 -1.43158721 -0.99393384 -1.92394725 -0.39216045  0.59255963
  0.81138631  0.04549292  0.59255963  0.70197297 -0.22804044  0.31902627
  0.42843962 -1.81453391  0.15490626 -0.44686713  0.86609299 -0.17333377
  0.48314629  0.37373294  0.86609299  0.2643196  -0.55628047  0.97550633
  0.37373294  0.20961293  0.20961293 -0.61098714  0.2643196   1.19433301
 -0.44686713 -1.48629388  0.04549292  0.59255963  0.31902627 -0.77510715
 -0.55628047 -3.51044072 -2.63513398  0.48314629  0.2643196  -0.1186271
 -0.00921376  0.53785296 -1.32217387  0.86609299  0.86609299 -0.00921376
  0.92079966  0.6472663   0.37373294 -0.72040048 -0.33745378 -3.12749402
 -1.37688054

In [46]:
# Regresión lineal (sin penalización) y p-values
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Tamaños
n, p = X_train_scaled.shape

# Predicciones y R2
y_hat_train = lr.predict(X_train_scaled)
R2_train = r2_score(y_train, y_hat_train)

y_hat_test = lr.predict(X_test_scaled)
R2_test = r2_score(y_test, y_hat_test)

# RSS y RSE (en train)
RSS = np.sum((y_train - y_hat_train)**2)
RSE = np.sqrt(RSS / (n - p - 1))

# Varianzas de beta (X incluye intercepto)
X_design = np.column_stack([np.ones(n), X_train_scaled.values])
XtX_inv = np.linalg.pinv(X_design.T @ X_design)
var_beta = XtX_inv * (RSE**2)
std_beta = np.sqrt(np.diag(var_beta))

# t-stats y p-values bilaterales
beta = np.r_[lr.intercept_, lr.coef_]
t_values = beta / std_beta
dfree = n - p - 1
p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df=dfree))

In [47]:
# Resultados Obtenidos
print("\n Resultados de la regresión sin penalización")
print("Intercepto:", lr.intercept_)

print("Columnas en X (orden):", X.columns.tolist())

# Coeficientes en el mismo orden de las columnas en X
print("Coeficientes:", lr.coef_)

# P-values:
print("P-values (orden: [Intercepto] + columnas en X):", p_values)

print("R2 (train):", R2_train)
print("R2 (test):", R2_test)
print("GL (n - p - 1):", dfree)


 Resultados de la regresión sin penalización
Intercepto: 8.086293260607736
Columnas en X (orden): ['Released_Year', 'Runtime', 'Meta_score', 'Drama', 'Crime', 'Action', 'Biography', 'Western', 'Comedy', 'Adventure', 'Animation', 'Horror', 'Mystery', 'Film-Noir', 'Fantasy', 'Family', 'Thriller', 'Romance', 'Sci-Fi', 'War', 'Music', 'Musical', 'Sport', 'History']
Coeficientes: [-0.02491257  0.10947437  0.08278779 -0.08783343 -0.02349929 -0.06416897
 -0.1170211  -0.08695303 -0.03068953 -0.0811902   0.05331447 -0.18043209
 -0.04768065 -0.04014471 -0.10320695 -0.06728095 -0.03929    -0.07610397
 -0.02258708  0.01335441  0.01687677 -0.09431525  0.06706191 -0.06154042]
P-values (orden: [Intercepto] + columnas en X): [0.00000000e+00 1.89319514e-01 1.19046406e-08 2.72850021e-06
 1.29134879e-01 6.18527557e-01 2.39808794e-01 3.56220861e-02
 4.96609870e-01 5.36687839e-01 1.57532985e-01 4.87374451e-01
 1.36002780e-01 4.33571256e-01 7.85031334e-01 1.65437965e-01
 4.65551318e-01 4.53614935e-01 1.563

In [50]:
# Regresión con Penalización L2 Ridge para datos de entrenamiento y prueba
ridge = Ridge(alpha=0.1, fit_intercept=True, random_state=42)
ridge.fit(X_train_scaled, y_train)

y_hat_train_ridge = ridge.predict(X_train_scaled)
y_hat_test_ridge  = ridge.predict(X_test_scaled)

R2_train_ridge = r2_score(y_train, y_hat_train_ridge)
R2_test_ridge  = r2_score(y_test,  y_hat_test_ridge)

# Resultados Obtenidos
print("Alpha:", 0.1)
print("Intercepto:", ridge.intercept_)
print("Columnas en X (orden):", X.columns.tolist())
print("Coeficientes:", ridge.coef_)
print("R2 (train):", R2_train_ridge)
print("R2 (test):",  R2_test_ridge)

Alpha: 0.1
Intercepto: 8.083907964092806
Columnas en X (orden): ['Released_Year', 'Runtime', 'Meta_score', 'Drama', 'Crime', 'Action', 'Biography', 'Western', 'Comedy', 'Adventure', 'Animation', 'Horror', 'Mystery', 'Film-Noir', 'Fantasy', 'Family', 'Thriller', 'Romance', 'Sci-Fi', 'War', 'Music', 'Musical', 'Sport', 'History']
Coeficientes: [-0.02471733  0.10932155  0.08282448 -0.08664818 -0.02280436 -0.06330282
 -0.11585437 -0.08450971 -0.029774   -0.08022625  0.05315185 -0.1763795
 -0.04724628 -0.03785613 -0.10190288 -0.06603951 -0.03868643 -0.07523967
 -0.02219015  0.01404562  0.01654033 -0.09150941  0.06690096 -0.06069455]
R2 (train): 0.2377255815931586
R2 (test): 0.09100758887883786


In [51]:
# Regresión con Penalización L1 Lasso para datos de entrenamiento y prueba
lasso = Lasso(alpha=0.1, fit_intercept=True, random_state=42, max_iter=10000)
lasso.fit(X_train_scaled, y_train)

y_hat_train_lasso = lasso.predict(X_train_scaled)
y_hat_test_lasso  = lasso.predict(X_test_scaled)

R2_train_lasso = r2_score(y_train, y_hat_train_lasso)
R2_test_lasso  = r2_score(y_test,  y_hat_test_lasso)

# Resultados Obtenidos
print("Alpha:", 0.1)
print("Intercepto:", lasso.intercept_)
print("Columnas en X (orden):", X.columns.tolist())
print("Coeficientes:", lasso.coef_)
print("R2 (train):", R2_train_lasso)
print("R2 (test):",  R2_test_lasso)


Alpha: 0.1
Intercepto: 7.931228070175439
Columnas en X (orden): ['Released_Year', 'Runtime', 'Meta_score', 'Drama', 'Crime', 'Action', 'Biography', 'Western', 'Comedy', 'Adventure', 'Animation', 'Horror', 'Mystery', 'Film-Noir', 'Fantasy', 'Family', 'Thriller', 'Romance', 'Sci-Fi', 'War', 'Music', 'Musical', 'Sport', 'History']
Coeficientes: [-0.  0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0.  0. -0. -0. -0. -0.
 -0.  0.  0.  0.  0.  0.]
R2 (train): 0.0
R2 (test): -0.0013103412373274281


In [52]:
# Comparación de modelos
comparacion = pd.DataFrame({
    "modelo": ["Ridge", "Lasso"],
    "alpha":  [0.1, 0.1],
    "R2_train": [R2_train_ridge, R2_train_lasso],
    "R2_test":  [R2_test_ridge,  R2_test_lasso]
})
print("\nComparación (Ridge vs Lasso)")
print(comparacion)



Comparación (Ridge vs Lasso)
  modelo  alpha  R2_train   R2_test
0  Ridge    0.1  0.237726  0.091008
1  Lasso    0.1  0.000000 -0.001310


## Interpretación de resultados

- **Regresión lineal sin penalización:**  
  - R² en train: **0.2377**  
  - R² en test: **0.0902**  
  Esto muestra que el modelo explica apenas ~24% de la variabilidad en entrenamiento y apenas ~9% en prueba. Es decir, su **capacidad predictiva es limitada** y su poder de generalización es bajo.  
  - Algunos p-values fueron muy significativos (ej. `Meta_score` con 1.19e-08 y `Drama` con 2.7e-06), lo que confirma que estas variables tienen un peso importante en explicar el IMDB Rating.  
  - Sin embargo, la mayoría de los géneros presentan p-values altos (mayores a 0.05), lo que significa que **no aportan significativamente** al modelo.  

- **Ridge (α = 0.1):**  
  - R² en train: **0.2377**  
  - R² en test: **0.0910**  
  Los resultados prácticamente no mejoran frente a la regresión simple, pero sí logran un pequeño ajuste positivo en el test. Ridge estabiliza los coeficientes (evita que se disparen), pero con este α bajo no se ve un cambio drástico en el poder predictivo.  
  - Interpretación: Ridge **no aumenta mucho la precisión**, pero sí controla mejor los coeficientes para dar un modelo más robusto.  

- **Lasso (α = 0.1):**  
  - R² en train: **0.0000**  
  - R² en test: **-0.0013**  
  Aquí el modelo prácticamente **no logra explicar nada**. El valor negativo en test indica que el modelo lo hace peor que simplemente usar la media como predicción.  
  - Interpretación: con este α, Lasso penalizó en exceso y llevó la mayoría de coeficientes a cero, dejando un modelo demasiado simple que perdió toda capacidad predictiva.  

### Conclusión
- El mejor resultado se dio con la regresión lineal sin penalización y con Ridge, aunque ambos muestran un poder explicativo bajo (R² < 0.25).  
- Las variables que realmente destacan son `Meta_score` y el género `Drama`, por su significancia estadística.  
- **Lasso no funcionó bien** en este caso con α = 0.1, porque eliminó demasiadas variables y el modelo se volvió irrelevante.  
- En general, los resultados indican que con este dataset y estas variables, el IMDB Rating es difícil de predecir con regresión lineal, y que el **Meta_score y ciertos géneros específicos** son los factores que más peso real tienen.  


## 5) Cierre y aprendizajes

Al hacer la correón del examen de regresión pude identificar varios puntos clave:

- **Preparación de datos:** me quedó claro que no basta con cargar el dataset; fue necesario limpiar duplicados, valores nulos y transformar columnas para que el modelo funcionara correctamente. Este paso evitó errores y sesgos.

- **Selección de variables:** aprendí a diferenciar entre variables útiles y las que solo agregan ruido (por ejemplo, títulos de películas o directores). Al quedarme con las más relevantes (año, duración, Meta_score y géneros), el modelo ganó claridad y estabilidad.

- **Escalamiento:** entendí la importancia de estandarizar variables numéricas cuando se aplican regresiones con penalización. Esto asegura que todas las variables tengan el mismo peso al momento de calcular coeficientes.

- **Modelos de regresión:** comprobé cómo la regresión lineal sin penalización puede sobreajustar, mientras que Ridge y Lasso ayudan a controlar la complejidad. Lasso además fuerza a que algunos coeficientes se vuelvan cero, lo que simplifica la interpretación.

- **Interpretación de métricas:** usar R² en train y test me permitió medir la capacidad de generalización y entender si el modelo estaba sobreajustado o no. También reforcé la utilidad de los p-values para evaluar la relevancia estadística de cada variable.



Que podemos hacer para mejorar el modelo:

**Generar nuevas variables (feature engineering):**  
  - Crear interacciones entre géneros (ej. `Drama*Romance`) para capturar combinaciones relevantes.  
  - Normalizar o agrupar géneros poco frecuentes para evitar ruido.  
  - Incluir transformaciones de `Runtime` (ej. categorizar películas cortas, medias, largas).  

- **Cambiar de modelo:**  
  Si el objetivo es predecir con mayor precisión, vale la pena probar métodos más flexibles como **Random Forests** o **Gradient Boosting**, que capturan relaciones no lineales.  

- **Validación cruzada:**  
  Evaluar el modelo con k-fold cross-validation en vez de un solo train/test split. Esto da una visión más estable del rendimiento real.  

- **Interpretabilidad:**  
  Para modelos lineales, usar gráficos de coeficientes y sus intervalos de confianza para visualizar qué variables aportan más. Esto ayuda a comunicar hallazgos de forma clara.  

En resumen: los resultados actuales son un buen punto de partida, pero muestran que el dataset es complejo y que **no basta con regresión lineal simple**. Con penalización adecuada, ingeniería de variables y modelos más flexibles, se puede mejorar la capacidad predictiva y obtener conclusiones más sólidas.

### Probemos con K-Folds:

In [53]:
# Validación cruzada con k-folds (Ridge y Lasso)
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge, Lasso
import numpy as np

# Configuración de los folds
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Ridge con cross-validation
alpha_ridge = 0.1
ridge = Ridge(alpha=alpha_ridge, fit_intercept=True, random_state=42)

scores_ridge = cross_val_score(ridge, X, y, cv=kf, scoring="r2")
print("Ridge (k-folds)")
print("Alpha:", alpha_ridge)
print("R² por fold:", scores_ridge)
print("R² promedio:", np.mean(scores_ridge))

# Lasso con cross-validation
alpha_lasso = 0.1
lasso = Lasso(alpha=alpha_lasso, fit_intercept=True, random_state=42, max_iter=10000)

scores_lasso = cross_val_score(lasso, X, y, cv=kf, scoring="r2")
print("\nLasso (k-folds)")
print("Alpha:", alpha_lasso)
print("R² por fold:", scores_lasso)
print("R² promedio:", np.mean(scores_lasso))

Ridge (k-folds)
Alpha: 0.1
R² por fold: [ 0.10434601  0.06744292  0.17099789 -0.08457293  0.19350266]
R² promedio: 0.09034330986752566

Lasso (k-folds)
Alpha: 0.1
R² por fold: [ 0.1358073   0.1148575   0.17898913 -0.04169844  0.18710143]
R² promedio: 0.11501138189481157


### Conclusión
- Ambos modelos muestran que el poder explicativo del dataset es limitado (R² promedio < 0.12).  
- **Lasso resultó ligeramente mejor** que Ridge en validación cruzada, lo que significa que la penalización que simplifica el modelo ayudó a generalizar un poco mejor.  
- Aún así, la variación entre folds (valores negativos y positivos) indica que **la capacidad de predicción es muy baja y dependiente del subconjunto de datos**.  

### Aprendizaje clave
La validación cruzada confirma que el modelo no es robusto: aunque algunos folds muestran R² ~0.18–0.19, otros caen negativos. Esto refuerza la idea de que **con las variables actuales no basta**, y sería necesario:
- **Probar más valores de α** para Ridge y Lasso (buscar un balance).  
- **Agregar nuevas variables** o features derivados.  
- **Explorar modelos no lineales** (árboles, boosting) que podrían captar mejor relaciones complejas.  
